Paquetes necesarios

In [1]:
import cv2  
import math 

from ultralytics import YOLO
from matplotlib import pyplot as plt

Extraemos las clases del modelo YOLO 11

In [ ]:
model = YOLO('yolo11n.pt')

vid = cv2.VideoCapture("C0142.MP4")

names = None

while vid.isOpened():
    ret, frame = vid.read()

    if ret:
        results = model(frame, show=False)
        if names is None:
            names = results[0].names
        annotated_frame = results[0].plot()
        cv2.imshow("Deteccion de YOLO", annotated_frame)

        # Salir del vídeo cuando presionamos ESC
        if cv2.waitKey(1) & 0xFF == 27 or cv2.getWindowProperty("Deteccion de YOLO", cv2.WND_PROP_VISIBLE) < 1:
            break
    else:
        # El vídeo ya se terminó
        break

vid.release()
cv2.destroyAllWindows()

# Leemos las posibles clases
with open("classes.txt", "w") as f:
    f.write(str(names))

### Mostramos el funcionamiento de nuestro modelo entrenado

In [ ]:
model = YOLO('best.pt')

vid = cv2.VideoCapture("C0142.MP4")

while vid.isOpened():
    ret, frame = vid.read()

    if ret:
        results = model(frame, show=False)
        annotated_frame = results[0].plot()
        cv2.imshow("Deteccion de YOLO", annotated_frame)

        # Salir del vídeo cuando presionamos ESC
        if cv2.waitKey(1) & 0xFF == 27 or cv2.getWindowProperty("Deteccion de YOLO", cv2.WND_PROP_VISIBLE) < 1:
            break
    else:
        # El vídeo ya se terminó
        break

vid.release()
cv2.destroyAllWindows()

### Usamos el modelo pre-entrenado de YOLO y el nuestro en conjunto 
Utilizamos el modelo pre-entrenado para detectar personas y vehículos, posteriormente, cuando hayamos detectado un vehículo, se lo pasamos a nuestro modelo entrenado en matrículas para que le detecte la matrícula.

In [6]:
base_model = YOLO('yolo11n.pt')
our_model = YOLO('best.pt')
vid = cv2.VideoCapture("C0142.MP4")

def get_coordinates(bounding_boxes, index):
    x_center = bounding_boxes[index][0]
    y_center = bounding_boxes[index][1]
    width = bounding_boxes[index][2]
    height = bounding_boxes[index][3]
    x = x_center-(width//2)
    y = y_center-(height//2)
    return int(x), int(y), int(width), int(height)

while vid.isOpened():
    ret, frame = vid.read()

    if ret:
        base_results = base_model(frame, show=False)
        plates_result = None
        annotated_frame = base_results[0].plot()
        bounding_boxes = list()
        names = list()
        for result in base_results:
            bounding_boxes += (result.boxes.xywh)
            names += [result.names[cls.item()] for cls in result.boxes.cls.int()]
            for i, name in enumerate(names):
                x1, y1, w1, h1 = get_coordinates(bounding_boxes, i)
                if name != "person":
                    vehicle_box = frame[y1:y1+h1, x1:x1+w1]
                    plates_result = our_model(vehicle_box, show=False)
                    for j, plate_details in enumerate(plates_result):
                        x2, y2, w2, h2 = get_coordinates(vehicle_box, j)
                        


        cv2.imshow("Deteccion de YOLO", annotated_frame)

        # Salir del vídeo cuando presionamos ESC
        if cv2.waitKey(1) & 0xFF == 27 or cv2.getWindowProperty("Deteccion de YOLO", cv2.WND_PROP_VISIBLE) < 1:
            break
    else:
        # El vídeo ya se terminó
        break

vid.release()
cv2.destroyAllWindows()


0: 384x640 4 cars, 1 bus, 124.5ms
Speed: 71.0ms preprocess, 124.5ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 544x640 (no detections), 193.6ms
Speed: 7.1ms preprocess, 193.6ms inference, 1.2ms postprocess per image at shape (1, 3, 544, 640)


TypeError: only length-1 arrays can be converted to Python scalars